# 추천 시스템 - 중요⭐

Data가 없는 경우  
-> 고객의 행동에서 특정 콘텐츠의 특징을 추출하여 다음 추천을 결정(콘텐츠 기반 필터링) - 정확도는 떨어짐     
  
Data가 쌓인 경우  
-> 고객들간의 지난 행동이 유사한 것이라 가정하여 다른 사람의 패턴과 유사함을 비교하여 추천(협업 필터링)

# 콘텐츠 기반 필터링 예측

In [45]:
import pandas as pd

raw_df = pd.read_csv('../data/tmdb_5000_movies.csv')
raw_df.head()

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""nam...",http://www.avata...,19995,"[{""id"": 1463, ""n...",en,Avatar,In the 22nd cent...,150.437577,"[{""name"": ""Ingen...","[{""iso_3166_1"": ...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""...",Released,Enter the World ...,Avatar,7.2,11800
1,300000000,"[{""id"": 12, ""nam...",http://disney.go...,285,"[{""id"": 270, ""na...",en,Pirates of the C...,Captain Barbossa...,139.082615,"[{""name"": ""Walt ...","[{""iso_3166_1"": ...",2007-05-19,961000000,169.0,"[{""iso_639_1"": ""...",Released,At the end of th...,Pirates of the C...,6.9,4500
2,245000000,"[{""id"": 28, ""nam...",http://www.sonyp...,206647,"[{""id"": 470, ""na...",en,Spectre,A cryptic messag...,107.376788,"[{""name"": ""Colum...","[{""iso_3166_1"": ...",2015-10-26,880674609,148.0,"[{""iso_639_1"": ""...",Released,A Plan No One Es...,Spectre,6.3,4466
3,250000000,"[{""id"": 28, ""nam...",http://www.theda...,49026,"[{""id"": 849, ""na...",en,The Dark Knight ...,Following the de...,112.312950,"[{""name"": ""Legen...","[{""iso_3166_1"": ...",2012-07-16,1084939099,165.0,"[{""iso_639_1"": ""...",Released,The Legend Ends,The Dark Knight ...,7.6,9106
4,260000000,"[{""id"": 28, ""nam...",http://movies.di...,49529,"[{""id"": 818, ""na...",en,John Carter,John Carter is a...,43.926995,"[{""name"": ""Walt ...","[{""iso_3166_1"": ...",2012-03-07,284139100,132.0,"[{""iso_639_1"": ""...",Released,Lost in our worl...,John Carter,6.1,2124


In [46]:
df = raw_df[['id', 'title', 'genres', 'vote_average', 'vote_count', 'popularity', 'keywords', 'popularity', 'overview']]
pd.set_option('max_colwidth', 20)
df

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,"[{""id"": 28, ""nam...",7.2,11800,150.437577,"[{""id"": 1463, ""n...",150.437577,In the 22nd cent...
1,285,Pirates of the C...,"[{""id"": 12, ""nam...",6.9,4500,139.082615,"[{""id"": 270, ""na...",139.082615,Captain Barbossa...
2,206647,Spectre,"[{""id"": 28, ""nam...",6.3,4466,107.376788,"[{""id"": 470, ""na...",107.376788,A cryptic messag...
3,49026,The Dark Knight ...,"[{""id"": 28, ""nam...",7.6,9106,112.312950,"[{""id"": 849, ""na...",112.312950,Following the de...
4,49529,John Carter,"[{""id"": 28, ""nam...",6.1,2124,43.926995,"[{""id"": 818, ""na...",43.926995,John Carter is a...
...,...,...,...,...,...,...,...,...,...
4798,9367,El Mariachi,"[{""id"": 28, ""nam...",6.6,238,14.269792,"[{""id"": 5616, ""n...",14.269792,El Mariachi just...
4799,72766,Newlyweds,"[{""id"": 35, ""nam...",5.9,5,0.642552,[],0.642552,A newlywed coupl...
4800,231617,"Signed, Sealed, ...","[{""id"": 35, ""nam...",7.0,6,1.444476,"[{""id"": 248, ""na...",1.444476,"""Signed, Sealed,..."
4801,126186,Shanghai Calling,[],5.7,7,0.857008,[],0.857008,When ambitious N...


In [47]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            4803 non-null   int64  
 1   title         4803 non-null   object 
 2   genres        4803 non-null   object 
 3   vote_average  4803 non-null   float64
 4   vote_count    4803 non-null   int64  
 5   popularity    4803 non-null   float64
 6   keywords      4803 non-null   object 
 7   popularity    4803 non-null   float64
 8   overview      4800 non-null   object 
dtypes: float64(3), int64(2), object(4)
memory usage: 337.8+ KB


In [48]:
from ast import literal_eval # 문자열을 자료형으로 변환(str -> list)

# 문자열 -> list, dict로 변환
# result = literal_eval(df.iloc[0]['genres']) # 각각 적용하여 결과 추출

df['genres'] = df['genres'].apply(literal_eval) # 함수 자체를 해당 컬럼에 적용
df['genres']

/var/folders/nl/dky9tsyn227718ljm_s34x600000gn/T/ipykernel_20018/523635665.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = df['genres'].apply(literal_eval) # 함수 자체를 해당 컬럼에 적용


0       [{'id': 28, 'nam...
1       [{'id': 12, 'nam...
2       [{'id': 28, 'nam...
3       [{'id': 28, 'nam...
4       [{'id': 28, 'nam...
               ...         
4798    [{'id': 28, 'nam...
4799    [{'id': 35, 'nam...
4800    [{'id': 35, 'nam...
4801                     []
4802    [{'id': 99, 'nam...
Name: genres, Length: 4803, dtype: object

In [49]:
df['keywords'] = df['keywords'].apply(literal_eval)
df['keywords']

/var/folders/nl/dky9tsyn227718ljm_s34x600000gn/T/ipykernel_20018/3086860884.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = df['keywords'].apply(literal_eval)


0       [{'id': 1463, 'n...
1       [{'id': 270, 'na...
2       [{'id': 470, 'na...
3       [{'id': 849, 'na...
4       [{'id': 818, 'na...
               ...         
4798    [{'id': 5616, 'n...
4799                     []
4800    [{'id': 248, 'na...
4801                     []
4802    [{'id': 1523, 'n...
Name: keywords, Length: 4803, dtype: object

In [50]:
total_names = []

for id_names in df['genres']:
    
    names = []
    
    for id_name in id_names:
        names.append(id_name['name'])
        
    total_names.append(' '.join(names))   
    
df['genres'] = total_names

/var/folders/nl/dky9tsyn227718ljm_s34x600000gn/T/ipykernel_20018/1897628688.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['genres'] = total_names


In [51]:
total_keywords = []

for id_keywords in df['keywords']:
    
    keywords = []
    
    for id_keyword in id_keywords:
        keywords.append(id_keyword['name'])
        
    total_keywords.append(' '.join(keywords))   
    
df['keywords'] = total_keywords 

/var/folders/nl/dky9tsyn227718ljm_s34x600000gn/T/ipykernel_20018/3119218250.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['keywords'] = total_keywords


In [52]:
# pandas select
print(df.loc[3, 'genres']) # 값(df.loc[3]['genres']도 가능하나 바로 가져오는게 아니라 상대적으로 느림)
print(df.iloc[3]['genres']) # 위치

Action Crime Drama Thriller
Action Crime Drama Thriller


In [53]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(min_df=1, ngram_range=(1,2)) # 최소 1번 이상 나오고, 2개씩 조합한 경우의 수를 벡터 행으로 활용

genres_vector = count.fit_transform(df['genres']) # 'comedy action'
keywords_vector = count.fit_transform(df['keywords']) # 'summer car'

count.vocabulary_ # 컬럼의 순서(횟수가 아님)

{'culture': 9037,
 'clash': 7010,
 'future': 15032,
 'space': 34485,
 'war': 40021,
 'colony': 7433,
 'society': 34214,
 'travel': 38236,
 'futuristic': 15087,
 'romance': 31412,
 'alien': 1077,
 'tribe': 38408,
 'planet': 27732,
 'cgi': 6161,
 'marine': 22347,
 'soldier': 34275,
 'battle': 3232,
 'love': 21496,
 'affair': 548,
 'anti': 1760,
 'power': 28328,
 'relations': 30072,
 'mind': 23178,
 'and': 1426,
 'soul': 34431,
 '3d': 94,
 'culture clash': 9039,
 'clash future': 7012,
 'future space': 15075,
 'space war': 34525,
 'war space': 40145,
 'space colony': 34494,
 'colony society': 7442,
 'society space': 34240,
 'space travel': 34523,
 'travel futuristic': 38253,
 'futuristic romance': 15094,
 'romance space': 31433,
 'space alien': 34487,
 'alien tribe': 1148,
 'tribe alien': 38409,
 'alien planet': 1122,
 'planet cgi': 27737,
 'cgi marine': 6165,
 'marine soldier': 22373,
 'soldier battle': 34281,
 'battle love': 3254,
 'love affair': 21498,
 'affair anti': 550,
 'anti war': 

In [54]:
genres_vector.shape # 전체 영화수, 장르의 조합수(벡터의 열수)

(4803, 276)

In [55]:
keywords_vector.shape # 전체 영화수, 키워드의 조합수(벡터의 열수)

(4803, 41554)

In [56]:
genres_vector_array = genres_vector.toarray() # 각 행이 각 영화
genres_vector_array

array([[1, 1, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [57]:
keywords_vector_array = keywords_vector.toarray()
keywords_vector_array

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity # 코사인 유사도로 유사한 행 확인

# 행 하나가 한 영화와 다른 영화가 유사한 코사인 유사도
genre_sim = cosine_similarity(genres_vector_array, genres_vector_array)
keyword_sim = cosine_similarity(keywords_vector_array, keywords_vector_array)
genre_sim

array([[1.        , 0.59628479, 0.4472136 , ..., 0.        , 0.        ,
        0.        ],
       [0.59628479, 1.        , 0.4       , ..., 0.        , 0.        ,
        0.        ],
       [0.4472136 , 0.4       , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [59]:
df.head()

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,Action Adventure...,7.2,11800,150.437577,culture clash fu...,150.437577,In the 22nd cent...
1,285,Pirates of the C...,Adventure Fantas...,6.9,4500,139.082615,ocean drug abuse...,139.082615,Captain Barbossa...
2,206647,Spectre,Action Adventure...,6.3,4466,107.376788,spy based on nov...,107.376788,A cryptic messag...
3,49026,The Dark Knight ...,Action Crime Dra...,7.6,9106,112.312950,dc comics crime ...,112.312950,Following the de...
4,49529,John Carter,Action Adventure...,6.1,2124,43.926995,based on novel m...,43.926995,John Carter is a...


## 추천 함수

In [ ]:
# 컨텐츠 기반 추출방식은 지양하는 것이 요즘 추세
def recommend_movies_by_genre(df, genre_sorted_index, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_index = genre_sorted_index[title_index, : (top_n)]
    similar_index = similar_index.reshape(-1)
    
    return df.iloc[similar_index]

# 값을 기준으로 오름차로 정렬하고 정렬 전 값의 index를 반환(argsort()), 이후 내림차로 변환([:,::-1]) => 값이 큰순으로 인덱스 반환
genre_sim_sorted = genre_sim.argsort()[:,::-1]

recommend_movies_by_genre(df, genre_sim_sorted, 'Superman', 5)

,id,title,genres,vote_average,vote_count,popularity,keywords,popularity,overview
0,19995,Avatar,Action Adventure...,7.2,11800,150.437577,culture clash fu...,150.437577,In the 22nd cent...
3494,27549,Beastmaster 2: T...,Action Adventure...,4.6,17,1.478505,based on novel t...,1.478505,Mark Singer retu...
813,1924,Superman,Action Adventure...,6.9,1022,48.507081,saving the world...,48.507081,Mild-mannered Cl...
870,8536,Superman II,Action Adventure...,6.5,629,30.515175,saving the world...,30.515175,Three escaped cr...
46,127585,X-Men: Days of F...,Action Adventure...,7.5,6032,118.078691,1970s mutant tim...,118.078691,The ultimate X-M...


In [ ]:
def recommend_movies_by_keyword(df, keyword_sorted_index, title_name, top_n=10):
    
    title_movie = df[df['title'] == title_name]
    title_index = title_movie.index.values
    
    similar_index = keyword_sorted_index[title_index, 1 : (top_n + 1)]
    similar_index = similar_index.reshape(-1) # [[  10 1296   14  870   65]] => [  10 1296   14  870   65]
    
    return df.iloc[similar_index]
    
keyword_sim_sorted = keyword_sim.argsort()[:,::-1]

test= recommend_movies_by_keyword(df, keyword_sim_sorted, 'Superman', 5)

[[  10 1296   14  870   65]]


# 협업 필터링 예측

In [62]:
import pandas as pd

movies = pd.read_csv('../data/movies.csv')
print(movies.head())
ratings = pd.read_csv('../data/ratings.csv')
ratings.head()


   movieId                title               genres
0        1     Toy Story (1995)  Adventure|Animat...
1        2       Jumanji (1995)  Adventure|Childr...
2        3  Grumpier Old Men...       Comedy|Romance
3        4  Waiting to Exhal...  Comedy|Drama|Rom...
4        5  Father of the Br...               Comedy


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


## 협업 필터링(유저 베이스 필터링)

In [91]:
mr = pd.merge(movies, ratings, on='movieId')
mr.head()

,movieId,title,genres,userId,rating,timestamp
0,1,Toy Story (1995),Adventure|Animat...,7,3.0,851866703
1,1,Toy Story (1995),Adventure|Animat...,9,4.0,938629179
2,1,Toy Story (1995),Adventure|Animat...,13,5.0,1331380058
3,1,Toy Story (1995),Adventure|Animat...,15,2.0,997938310
4,1,Toy Story (1995),Adventure|Animat...,19,3.0,855190091


In [92]:
title_user = mr.pivot_table(index='userId', columns='title', values='rating') # index가 행
title_user.head()

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [94]:
title_user.fillna(0, inplace=True) # -1로 채우는 경우 벡터의 방향성이 생기므로 0으로 처리

In [ ]:
user_based_result = cosine_similarity(title_user, title_user) # 사용자간 영화별 평점의 유사도를 구함
user_based_result
# title_based_result = cosine_similarity(title_user.T, title_user.T) # 영화간 평점 유사도


array([[1.        , 0.        , 0.        , ..., 0.06291708, 0.        ,
        0.01746565],
       [0.        , 1.        , 0.12429498, ..., 0.02413984, 0.17059464,
        0.1131753 ],
       [0.        , 0.12429498, 1.        , ..., 0.08098382, 0.13660585,
        0.17019275],
       ...,
       [0.06291708, 0.02413984, 0.08098382, ..., 1.        , 0.04260878,
        0.08520194],
       [0.        , 0.17059464, 0.13660585, ..., 0.04260878, 1.        ,
        0.22867673],
       [0.01746565, 0.1131753 , 0.17019275, ..., 0.08520194, 0.22867673,
        1.        ]])

In [67]:
user_based_df = pd.DataFrame(user_based_result, index=title_user.index, columns=title_user.index)
user_based_df

userId,1,2,3,4,5,6,7,8,9,10,...,662,663,664,665,666,667,668,669,670,671
userId,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.000000,0.000000,0.074482,0.016818,0.000000,0.083884,0.000000,0.012843,0.000000,...,0.000000,0.000000,0.014481,0.043719,0.000000,0.000000,0.000000,0.062917,0.000000,0.017466
2,0.000000,1.000000,0.124295,0.118821,0.103646,0.000000,0.212985,0.113190,0.113333,0.043213,...,0.477306,0.063202,0.077784,0.164162,0.466281,0.425462,0.084646,0.024140,0.170595,0.113175
3,0.000000,0.124295,1.000000,0.081640,0.151531,0.060691,0.154714,0.249781,0.134475,0.114672,...,0.161205,0.064198,0.176222,0.158357,0.177098,0.124562,0.124911,0.080984,0.136606,0.170193
4,0.074482,0.118821,0.081640,1.000000,0.130649,0.079648,0.319745,0.191013,0.030417,0.137186,...,0.114319,0.047228,0.136647,0.254030,0.121905,0.088735,0.068483,0.104309,0.054512,0.211609
5,0.016818,0.103646,0.151531,0.130649,1.000000,0.063796,0.095888,0.165712,0.086616,0.032370,...,0.191029,0.021142,0.146246,0.224245,0.139721,0.058252,0.042926,0.038358,0.062642,0.225086
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
667,0.000000,0.425462,0.124562,0.088735,0.058252,0.000000,0.232051,0.069005,0.066412,0.032653,...,0.342283,0.050754,0.074118,0.124214,0.351207,1.000000,0.091597,0.018416,0.153111,0.127995
668,0.000000,0.084646,0.124911,0.068483,0.042926,0.019563,0.058773,0.112366,0.194493,0.098561,...,0.074089,0.059010,0.093068,0.082525,0.114487,0.091597,1.000000,0.000000,0.178017,0.135387
669,0.062917,0.024140,0.080984,0.104309,0.038358,0.024583,0.073151,0.055143,0.029291,0.060549,...,0.015960,0.025953,0.077966,0.101707,0.028773,0.018416,0.000000,1.000000,0.042609,0.085202


In [ ]:
# 0번째 사용자와 가장 유사한 평점을 매긴 5명의 id
user_based_df.iloc[0].sort_values(ascending=False)[1:6]

userId
325    0.371852
634    0.194093
341    0.162819
310    0.157524
207    0.152746
Name: 1, dtype: float64

In [ ]:
# 1. 내가 본것 

my_list = []

ratings = title_user.iloc[0]

for idx, rating in enumerate(ratings): # index / 평점
    
    if rating > 0:
        my_list.append(title_user.iloc[0].index[idx]) # title_user.iloc[0]는 Series, idx로 영화명 가져옴
    
print(my_list)    
print(len(my_list))

# 2. 상대가 본 것 중에 평점 4.0이상

you_list = []

# loc(값비교) iloc(index비교) 
# 325를 넣었을때 => iloc로 324번째 값 / loc는 userId가 325인 값
ratings = title_user.loc[325] 
you_list = ratings[ratings>=4].index

print(you_list)
print(len(you_list))

['Antz (1998)', 'Beavis and Butt-Head Do America (1996)', 'Ben-Hur (1959)', 'Blazing Saddles (1974)', 'Cape Fear (1991)', 'Cinema Paradiso (Nuovo cinema Paradiso) (1989)', 'Dangerous Minds (1995)', 'Deer Hunter, The (1978)', "Dracula (Bram Stoker's Dracula) (1992)", 'Dumbo (1941)', 'Escape from New York (1981)', 'Fly, The (1986)', 'French Connection, The (1971)', 'Gandhi (1982)', 'Gods Must Be Crazy, The (1980)', 'Sleepers (1996)', 'Star Trek: The Motion Picture (1979)', 'Time Bandits (1981)', 'Tron (1982)', 'Willow (1988)']
20
Index(['Beverly Hills Cop (1984)', 'Brady Bunch Movie, The (1995)',
       'Dangerous Minds (1995)', 'My Best Friend's Wedding (1997)'],
      dtype='object', name='title')
4


In [70]:
# 3. 차집합으로 상대방은 봤는데 내가 안본 영화 확인
set(you_list) - set(my_list)

{'Beverly Hills Cop (1984)',
 'Brady Bunch Movie, The (1995)',
 "My Best Friend's Wedding (1997)"}

## 협업 필터링(콘텐츠 베이스 필터링)

In [71]:
content_base = title_user.T # 기존 데이터의 행렬 전환

print(content_base)

content_based_sim = cosine_similarity(content_base, content_base)
content_based_df = pd.DataFrame(content_based_sim, index=content_base.index, columns=content_base.index)

content_based_df


userId                1    2    3    4    5    6    7    8    9    10   ...  \
title                                                                   ...   
"Great Performanc...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
$9.99 (2008)          0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
'Hellboy': The Se...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
'Neath the Arizon...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
'Round Midnight (...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
...                   ...  ...  ...  ...  ...  ...  ...  ...  ...  ...  ...   
xXx (2002)            0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
xXx: State of the...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
¡Three Amigos! (1...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
À nous la liberté...  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  ...   
İtirazım Var (2014)   0.0  0.0  0.0  0.0  0.0  0.0  

title,"""Great Performances"" Cats (1998)",$9.99 (2008),'Hellboy': The Seeds of Creation (2004),'Neath the Arizona Skies (1934),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),...,Zulu (1964),Zulu (2013),[REC] (2007),eXistenZ (1999),loudQUIETloud: A Film About the Pixies (2006),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931),İtirazım Var (2014)
title,,,,,,,,,,,,,,,,,,,,,
"""Great Performances"" Cats (1998)",1.000000,0.000000,0.0,0.164399,0.020391,0.0,0.014046,0.000000,0.000000,0.003166,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000
$9.99 (2008),0.000000,1.000000,0.0,0.000000,0.000000,0.0,0.000000,0.079474,0.000000,0.156330,...,0.000000,0.000000,0.0,0.000000,0.0,0.013899,0.000000,0.058218,0.0,0.0000
'Hellboy': The Seeds of Creation (2004),0.000000,0.000000,1.0,0.000000,0.000000,1.0,0.000000,0.217357,0.000000,0.000000,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000
'Neath the Arizona Skies (1934),0.164399,0.000000,0.0,1.000000,0.124035,0.0,0.085436,0.000000,0.000000,0.019259,...,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0000
'Round Midnight (1986),0.020391,0.000000,0.0,0.124035,1.000000,0.0,0.010597,0.143786,0.000000,0.136163,...,0.000000,0.000000,0.0,0.121567,0.0,0.000000,0.000000,0.000000,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xXx (2002),0.000000,0.013899,0.0,0.000000,0.000000,0.0,0.000000,0.123940,0.000000,0.144961,...,0.161281,0.076029,0.0,0.017465,0.0,1.000000,0.152057,0.140222,0.0,0.2661
xXx: State of the Union (2005),0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.134815,...,0.000000,0.000000,0.0,0.000000,0.0,0.152057,1.000000,0.000000,0.0,0.0000
¡Three Amigos! (1986),0.000000,0.058218,0.0,0.000000,0.000000,0.0,0.081620,0.331663,0.214498,0.064908,...,0.112588,0.159223,0.0,0.166622,0.0,0.140222,0.000000,1.000000,0.0,0.0000


In [72]:
# 유저 기반으로 나온 결과(영화명)를 가지고, 컨텐츠 기반으로 다시 추천하는 것도 방법
content_based_df['Toy Story (1995)'].sort_values(ascending=False)[1:6] 

title
Toy Story 2 (1999)                           0.594710
Star Wars: Episode IV - A New Hope (1977)    0.576188
Forrest Gump (1994)                          0.564534
Independence Day (a.k.a. ID4) (1996)         0.562946
Groundhog Day (1993)                         0.548023
Name: Toy Story (1995), dtype: float64

# 서프라이즈 활용 평점 예측
### 사용자가 미사용한 상품에 대해 평점을 예상하고 높은 순으로 추천

In [ ]:
from surprise import SVD # C++ 설치 필요 / "numpy<2" / python 3.11 or 3.10
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import Reader
from surprise.dataset import DatasetAutoFolds

import pandas as pd

In [74]:
ratings = pd.read_csv('../data/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [ ]:
ratings.to_csv('../data/ratings_nohead.csv', index=False, header=False) # header=False로 header 제거하고 저장
ratings.describe() # 최소는 0이 아니라 0.5(안본 것(0)과 구분하기 위함)


,userId,movieId,rating,timestamp
count,100004.000000,100004.000000,100004.000000,1.000040e+05
mean,347.011310,12548.664363,3.543608,1.129639e+09
std,195.163838,26369.198969,1.058064,1.916858e+08
min,1.000000,1.000000,0.500000,7.896520e+08
25%,182.000000,1028.000000,3.000000,9.658478e+08
50%,367.000000,2406.500000,4.000000,1.110422e+09
75%,520.000000,5418.000000,4.000000,1.296192e+09
max,671.000000,163949.000000,5.000000,1.476641e+09


In [ ]:
ratings['rating'].unique()


,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [102]:
# line_format(Reader는 첫라인부터 값으로 인식하므로 설정), sep(구분자 : 기본값 ',')
movie_reader = Reader(line_format='user item rating timestamp', sep=',', rating_scale=(0.5, 5)) 
data_folds = DatasetAutoFolds(ratings_file='../data/ratings_nohead.csv', reader=movie_reader)

# 100% 다 훈련용으로 사용
trainset = data_folds.build_full_trainset() 

model = SVD(n_factors=50, random_state=42) # n_factors - 차원수
model.fit(trainset)

In [103]:
y_pred = model.predict('9', '3') # 9번 유저가 3번 영화를 보면 3.09점(est)을 줄 것으로 예상
y_pred

Prediction(uid='9', iid='3', r_ui=None, est=3.0923245412020983, details={'was_impossible': False})

In [104]:
# 1. 해당 유저가 안본 영화를 확인
movies = pd.read_csv('../data/movies.csv')
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9125 entries, 0 to 9124
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  9125 non-null   int64 
 1   title    9125 non-null   object
 2   genres   9125 non-null   object
dtypes: int64(1), object(2)
memory usage: 214.0+ KB


In [108]:
# 해당 유저가 본 영화
ratings[ratings['userId'] == 9]['movieId']

699       1
700      17
701      26
702      36
703      47
704     318
705     497
706     515
707     527
708     534
709     593
710     608
711     733
712    1059
713    1177
714    1357
715    1358
716    1411
717    1541
718    1584
719    1680
720    1682
721    1704
722    1721
723    1784
724    2028
725    2125
726    2140
727    2249
728    2268
729    2273
730    2278
731    2291
732    2294
733    2302
734    2391
735    2396
736    2427
737    2490
738    2501
739    2539
740    2571
741    2628
742    2762
743    2857
Name: movieId, dtype: int64

In [109]:
def get_unseen_movies(userId):
    return list(set(movies['movieId']) - set(ratings[ratings['userId'] == userId]['movieId']))

unseen_movies = get_unseen_movies(9)

In [120]:
pred_list = []

for movieId in unseen_movies:
    y_pred = model.predict('9', str(movieId))
    pred_list.append(
        {
            'movieId': y_pred.iid, 
            'rating': y_pred.est, 
            # Series에서 행번호 제외하고, 값만 빼기 위해 iloc(0)
            'title': movies[movies['movieId'] == int(y_pred.iid)]['title'].iloc[0] 
        }
    )

print(pred_list[0])

{'movieId': '2', 'rating': 3.4150971107531425, 'title': 'Jumanji (1995)'}


In [ ]:
result_df = pd.DataFrame(pred_list)
result_df.sort_values(by='rating', ascending=False)[:5]

9080


,movieId,rating,title
849,969,4.498787,"African Queen, T..."
752,858,4.453779,"Godfather, The (..."
795,913,4.450571,"Maltese Falcon, ..."
1066,1221,4.434328,Godfather: Part ...
2983,3462,4.390097,Modern Times (1936)
